### This Notebook controls the arima model for our analysis.

**The model is stored in the "arima_model.py" file, but it runs from this notebook.**

**Requirements:** A folder named "model_dump" which will be where models are exported.

**Import Modules**

In [ ]:
import importlib
import pandas as pd
import arima_model as arm
import data_getter as data_con

importlib.reload(arm) # Simply reloads the module in case of changes.

**This code block retrieves data from the database and then does preliminary preparation.**

In [ ]:
all_df = data_con.run_queries()
master_table = data_con.join_tables(all_df)

**Create models**
> This cell block loops through all the ages, and all the locations to create a model for each<br>
>and then preserve that model as well as the predicted data. The model will be exported for use in<br>
>the dashboard, while the predicted data is read into the dashboard for total area results (because <br>
>it would take too long to get the data for visuals that need quick updates.)

In [ ]:
# Create a list to hold all the prediction dataframes
prediction_dataframes = []

# Get the different possible age ranges
age_ranges = master_table.AgeGroup.unique().tolist()
age_ranges = [_ for _ in age_ranges if str(_) != 'nan']

# Get the different possible locations
all_fips = master_table.FIPS.unique().tolist()

# Determine number of runs that will occur
total = len(all_fips) * len(age_ranges)

print('\nApproximate Run Time: 90 minutes\n')
cntr = 0
for i, age_range in enumerate(age_ranges):
    for ii, target_fips in enumerate(all_fips):
        params = [target_fips, age_range]
        target = 'MedianIncome'
        
        try:
            # Send the data to the 'arima_model.py' "dispatcher" function which controls the logic
            graph_ready, export_ready = arm.dispatcher(master_table, target, params)
            
        except Exception as E:
            print(f'Failure: {E}, params: {params}')
            cntr += 1
            continue
        
        # fill out the age range of the new dataframe
        export_ready['AgeGroup'] = age_range
        
        # add the prediction dataframe to the dataframe list
        prediction_dataframes.append(export_ready)
        cntr += 1
        print(f'{cntr} of {total} completed')

**Combine all the prediction dataframes into a master dataframe.**
>This is then exported for use in visualizing overall results.

In [ ]:
main_frame = pd.DataFrame()
for frame in prediction_dataframes:
    frame['FIPS'] = frame['FIPS'].tolist()[0]
    main_frame = pd.concat([main_frame, frame])

# Export the prediction results for fast access to all prediction data
# that would otherwise take a long time for overall analysis
main_frame.to_csv('PredictedIncomeAfterFinalRun.csv',index = False)